## Basic ipynb to start

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
emotion_train = load_dataset('emotion',split='train')

In [ ]:
print(emotion_train)

In [ ]:
emotion_df = pd.DataFrame(emotion_train)
emotion_df.head()

In [ ]:
true_label = pd.DataFrame(emotion_df.label)
emotion_text_df = pd.DataFrame(emotion_df.text)
emotion_text_df.head()

## Define our Labeling Functions

In [ ]:
from snorkel.labeling import labeling_function,PandasLFApplier,LFAnalysis


## Ways to create func
- Keyword Lookup
- Pattern Lookup/Regex
- 3rd Party Library:spacy
- Suggestions...

In [ ]:
# Constants for our labels
ABSTAIN = -1
SADNESS = 0
JOY = 1
LOVE = 2
ANGER = 3
FEAR = 4
SUPRISE = 5


In [ ]:
## Keyword Lookup
@labeling_function()
def lf_sad_keyword_lookup(x):
  keywords = ["sad","low"]
  return SADNESS if any(word in x.text.lower() for word in keywords) else ABSTAIN

In [ ]:
## Pattern Lookup/Regex Fxn
import re
@labeling_function()
def lf_regex_contains_sad(x):
  return SADNESS if re.search(r"sad*.*",x.text.lower(),flags=re.I) else ABSTAIN

In [ ]:
### Pattern Lookup/Regex Fxn
@labeling_function()
def lf_regex_contains_happy(x):
  return JOY if re.search(r"happ*.*",x.text.lower(),flags=re.I) else ABSTAIN

In [ ]:
### Apply Fxn
lfs = [lf_sad_keyword_lookup,lf_regex_contains_sad,lf_regex_contains_happy]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=emotion_text_df)

In [ ]:
L_train

In [ ]:
### Evaluate the Performance
LFAnalysis(L=L_train,lfs=lfs).lf_summary()

#### We need to ensemble the votes and for this we can use MajorityLabel Voter by snorkel

In [ ]:
# LabelModel
#from snorkel.labeling.model import MajorityLabelVoter,LabelModel